# Forecast evaluation code

## Imports

In [19]:
# Imports
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from joblib import Parallel, delayed

## Non-linear models

In [20]:
def TR_model(S_t, gamma):
    # Return a vector of outcomes of the TR model
    return (S_t >= gamma).astype(float).reshape(-1, 1)

def LSTR_model(S_t,gamma,tau):
    # Return a vector of outcomes of the LSTR model
    return ((1+np.exp(-tau * (S_t-gamma)))**-1).reshape(-1, 1)

def ESTR_model(S_t,gamma,tau):
    # Return a vector of outcomes of the ESTR model
    return (1-np.exp(-tau * (S_t-gamma)**2)).reshape(-1, 1)

In [21]:
def nonlin_model(model_type, S_t, gamma, tau):
    
    # Detect which model type it should use according to model_type
    if (model_type == "TR"):
        return TR_model(S_t,gamma)
        
    elif (model_type == "LSTR"):
        return LSTR_model(S_t,gamma,tau)
        
    elif (model_type == "ESTR"):
        return ESTR_model(S_t,gamma,tau)
        
    # If we pass another model that is not included, we give an error
    else:
        raise Exception("Model type unknown")

## Test statistics

In [27]:
# Simple method to estimate the OLS estimates
def estimateBeta(Z_t,Y_t):
    return np.linalg.inv(np.transpose(Z_t) @ Z_t) @ np.transpose(Z_t) @ Y_t

In [25]:
def cal_g_theta(S_t, # Vector of S_t
                L_t, # Vector of loss function
                R_size, # Integer
                P_size, # Integer
                h_size, # Integer
                model_type # String of model type
                ):
    
    # Determine the size of B
    B_size = round(4*(P_size/100)**(2/9)+1)
    
    # Simulate a matrix of the standard normal distribution
    v_t = np.random.normal(0,1,(P_size+B_size,iterations_CV))
    
    # Initialize the critical values with a minimum value 
    crit_values = np.full(iterations_CV,-100)

    # Check which model we should use;
    if (model_type == "TR"):
        
        # Initialize the array for the W_P's
        W_P = np.zeros(grid_elements)
        
        # Create an array of gamma value that are used in the for loop
        gamma_quantile = np.linspace(0.15,0.85,grid_elements)
    
        # Loop over every parameter value to get the W_P's and the updated critical values 
        for i in range(grid_elements):
            W_P[i], W_P_j = cal_test_statistic(S_t, L_t, R_size, P_size, h_size, B_size, v_t, model_type, np.quantile(S_t,gamma_quantile[i]))
            crit_values = np.maximum(crit_values, W_P_j)

    else:
        
        # Initialize the array for the W_P's
        W_P = np.zeros(grid_elements**2)
        
        # Create an array of gamma and tau values that are used in the for loop
        gamma_quantile = np.linspace(0.15,0.85,grid_elements)
        tau = np.linspace(0.1,5,grid_elements)
        
        # Loop over every parameter value to get the W_P's and the updated critical values
        for i in range(grid_elements):
            for j in range(grid_elements):
                W_P[i*grid_elements+j], W_P_j = cal_test_statistic(S_t, L_t, R_size, P_size, h_size, B_size, v_t, model_type, np.quantile(S_t,gamma_quantile[i]),tau[j])
                crit_values = np.maximum(crit_values, W_P_j)
    
    # Get the test statistic of all W_P values
    g_theta = max(W_P)
    
    # Sort the Monte Carlo simulations and get critical value
    crit_values.sort()
    final_crit_value = crit_values[round(0.95 * iterations_CV)-1]

    # Check if the test statistic 
    if (g_theta > final_crit_value):
        return 1
    else:
        return 0

In [32]:
def cal_test_statistic(S_t, #vector of S_t
                       L_t, #vector of loss function
                       R_size, #integer
                       P_size, #integer
                       h_size, #integer
                       B_size, #integer
                       v_t, #matrix of standard normal distribution
                       model_type, #String of model type
                       gamma, #gamma parameter
                       tau=0  #tau parameter
                       ):
        
    # Initialize the matrix of lambda_sum, V_sum, M_sum
    lambda_sum = np.zeros((2,iterations_CV))
    V_sum = np.zeros((2,2))
    M_sum = np.zeros((2,2))
    
    # Make a vector of ones for X_t
    X_t = np.ones((len(S_t),1))
    
    # Get psi (scalar)
    Q_t = np.column_stack((X_t,np.multiply(X_t,nonlin_model(model_type,S_t,gamma,tau))))
    psi = np.column_stack(estimateBeta(Q_t[R_size-h_size:R_size+P_size-h_size],L_t))

    # Loop over t
    for t in range(R_size,R_size+P_size):

        # Select X, S, Y from a certain time range
        X_sel = X_t[t-R_size:t]
        S_sel = S_t[t-R_size:t]
        L_sel = np.column_stack(L_t[t-R_size])        
        
        # Get Q_sel
        Q_sel = Q_t[t-R_size:t]

        # Get residuals
        residual_sel = L_sel - np.column_stack(Q_sel[-1]) @ np.transpose(psi)

        # Get score
        score = np.transpose(np.column_stack(Q_sel[-1])) * residual_sel
        
        # Get V and M sum
        V_sum += score * np.transpose(score)
        M_sum += Q_sel[-1] * np.transpose(np.column_stack(Q_sel[-1]))

        # Loop over all B
        for b in range(0,B_size+1):
            lambda_sum += score * v_t[t+b-R_size,:]
            
    # Set Hr to I2
    Hr = np.identity(2)

    # Get V_P and M_P
    V_P = V_sum / P_size
    M_P = M_sum / P_size

    # Get Vstar
    V_star = np.linalg.inv(M_P) @ V_P @ np.linalg.inv(M_P)

    # Get Wp
    W_P = P_size * (psi @ Hr @ np.linalg.inv(np.transpose(Hr) @ V_star @ Hr) @ np.transpose(Hr) @ np.transpose(psi))

    # Get W_P_j
    lambda_P = (lambda_sum / ((P_size*(1+B_size))**0.5)).T
    W_P_j = np.sum(np.matmul(lambda_P, np.linalg.inv(M_P) @ Hr @ np.linalg.inv(np.transpose(Hr) @ V_star @ Hr) @ np.transpose(Hr) @ np.linalg.inv(M_P)) * lambda_P,axis=1)
    
    # Return the test statistic and critical values
    return W_P, W_P_j